In [70]:
import os, sys
from utils import *
from download_DB import download_db
import glob
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

from datetime import date
hoje =  pd.to_datetime(date.today())

In [71]:
files = []
for file in glob.glob("./Queries/v*.txt",):
    a,b = os.path.split(file)
    files.append(b.split(".")[0])
directory = download_db(lista_download=files)


******************** Baixando arquivos *********************

Carregando arquivo de log...
vmonitoramentoleilao já foi baixado hoje: Dia: 18/01/23 - Horário: 09:51:54.  Portanto não foi baixado novamente.
vmonitoramentoug já foi baixado hoje: Dia: 18/01/23 - Horário: 09:54:41.  Portanto não foi baixado novamente.
vmonitoramentousina já foi baixado hoje: Dia: 18/01/23 - Horário: 09:54:45.  Portanto não foi baixado novamente.
vrapeelacesso já foi baixado hoje: Dia: 18/01/23 - Horário: 09:54:54.  Portanto não foi baixado novamente.
vrapeelcontratorecurso já foi baixado hoje: Dia: 18/01/23 - Horário: 09:55:18.  Portanto não foi baixado novamente.
vrapeelcronograma já foi baixado hoje: Dia: 18/01/23 - Horário: 09:56:25.  Portanto não foi baixado novamente.
vrapeelempreendimento já foi baixado hoje: Dia: 18/01/23 - Horário: 09:56:35.  Portanto não foi baixado novamente.
vrapeellicenciamento já foi baixado hoje: Dia: 18/01/23 - Horário: 09:57:08.  Portanto não foi baixado novamente.
vrapeelo

In [72]:
# Lista de colunas que serão usadas várias vezes para a realização de merges entre os dataframes
ide_data_cols = ['DthEnvio','IdeUsinaOutorga']
ide_ug = ['IdeUsinaOutorga','NumOperacaoUg']

# Baixa bancos de dados

In [73]:
# Carrega BD vmonitoramentoleilao => monitoramentoleilao
monitoramentoleilao  = pd.read_parquet(os.path.join(directory,"vmonitoramentoleilao.gzip"))[['IdeUsinaOutorga','CodLeilao', 'DatInicioSuprimento']]
monitoramentoleilao.dropna(subset='DatInicioSuprimento',inplace=True)
monitoramentoleilao.sample(5)

,IdeUsinaOutorga,CodLeilao,DatInicioSuprimento
2275,49670,06/2021,2024-01-01
1011,37959,03/2018,2024-01-01
681,35830,02/2016,2017-01-01
1447,44605,01/2019,2021-06-28
3038,56130,01/2021,2022-05-01


In [74]:
# Carrega BD vrapeelcronograma => rapeel_cronograma
cols = ['IdeUsinaOutorga','DthEnvio','DatRealizacaoII','DatPrevistaAprovacaoIII','DatRealizacaoIII','DatRealizacaoIX','DatRealizacaoXI','DatRealizacaoXIII','DatRealizacaoXIV','DatRealizacaoXII','DatRealizacaoVII','DatRealizacaoVIII','DatRealizacaoVI','DatRealizacaoX','DatRealizacaoV','DatRealizacaoIV']
rename_cols = {
        'DthEnvio' : 'Data_Envio',
        'NomUsina' : 'usina_nome',
        'DatRealizacaoII' : 'canteiroReal',
        'DatPrevistaAprovacaoIII' : 'prev_IO',
        'DatRealizacaoIII' : 'IO_real',
        'DatRealizacaoIX' : 'DesvRio_real',
        'DatRealizacaoXI' : 'Ench_Real',
        'DatRealizacaoXIII' : 'IOTrans_Real',
        'DatRealizacaoXIV' : 'Conc_Trans_REAL',
        'DatRealizacaoXII' : 'Comiss_Real',
        'DatRealizacaoVII' : 'ME_Real_conc_eol',
        'DatRealizacaoVIII' : 'Conc_Combust_Real'}
        
rapeel_cronograma = pd.read_parquet(os.path.join(directory,"vrapeelcronograma.gzip"))[cols].rename(columns=rename_cols)
rapeel_cronograma.sample(5)

,IdeUsinaOutorga,Data_Envio,canteiroReal,prev_IO,IO_real,DesvRio_real,Ench_Real,IOTrans_Real,Conc_Trans_REAL,Comiss_Real,ME_Real_conc_eol,Conc_Combust_Real,DatRealizacaoVI,DatRealizacaoX,DatRealizacaoV,DatRealizacaoIV
33755,50807,2022-10-04 11:29:28.260,NaT,2024-07-22,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
40409,50529,2023-01-04 13:16:26.493,NaT,2025-01-30,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
31580,48640,2022-09-02 10:16:30.473,NaT,2022-09-26,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
10454,49177,2021-04-05 23:57:06.807,NaT,2022-01-31,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
7253,32211,2021-01-04 16:20:31.047,2020-06-18,NaT,2020-07-06,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT


In [75]:
# Carrega BD vrapeelcontratorecurso => mercado

cols = ['DthEnvio','IdeUsinaOutorga','DatConclusaoIII',]
rename_cols = {'DatConclusaoIII':'DatConclusaoACL'}
mercado = pd.read_parquet(os.path.join(directory,"vrapeelcontratorecurso.gzip"))[cols].rename(columns=rename_cols)

# Carrega BD vrapeellicenciamento => ambiental_datas
cols = ["DatValidadeIV","DatValidadeI","DthEnvio","IdeUsinaOutorga","DatValidadeV"]
rename_cols = {'DatValidadeIV': 'DatValidadeLI','DatValidadeI': 'DatValidadeLP','DatValidadeV': 'DatValidadeLO'}
ambiental_datas = pd.read_parquet(os.path.join(directory,"vrapeellicenciamento.gzip"))[cols].rename(columns=rename_cols)
ambiental_datas = ambiental_datas.loc[ambiental_datas.groupby('IdeUsinaOutorga').DthEnvio.idxmax()]

# Carrega BD vrapeelacesso => acesso_contratos
cols = ["DthEnvio","NomSitContratoI","NomSitContratoIV","NomSituacaoContratoIII","NomSitContratoII","IdeUsinaOutorga"]
rename_cols = {'NomSitContratoI': 'NomSitContratoCCD','NomSitContratoIV': 'NomSitContratoCUST','NomSituacaoContratoIII': 'NomSituacaoContratoCUSD','NomSitContratoII': 'NomSitContratoCCT'}
acesso_contratos = pd.read_parquet(os.path.join(directory,"vrapeelacesso.gzip"))[cols].rename(columns=rename_cols)
acesso_contratos = acesso_contratos.loc[acesso_contratos.groupby('IdeUsinaOutorga').DthEnvio.idxmax()]

# Carrega BD vrapeeloperacaoug => ug_rapeel
cols = ["IdeUsinaOutorga",'NumOperacaoUg',"DthEnvio",'DatPrevistaComercial']
ug_rapeel = pd.read_parquet(os.path.join(directory,"vrapeeloperacaoug.gzip"))[cols]
ug_rapeel = ug_rapeel.loc[ug_rapeel.groupby(['IdeUsinaOutorga','NumOperacaoUg']).DthEnvio.idxmax()]

# Cria dataframe input_rapeel

In [76]:
# Carrega BD vrapeelacesso => acesso_contratos
input_rapeel = pd.merge(mercado,ambiental_datas,on=ide_data_cols,how="inner")
input_rapeel = pd.merge(input_rapeel,acesso_contratos,on=ide_data_cols,how="inner")
input_rapeel = input_rapeel.loc[input_rapeel.groupby('IdeUsinaOutorga').DthEnvio.idxmax()]
input_rapeel.sample(5)

,DthEnvio,IdeUsinaOutorga,DatConclusaoACL,DatValidadeLI,DatValidadeLP,DatValidadeLO,NomSitContratoCCD,NomSitContratoCUST,NomSituacaoContratoCUSD,NomSitContratoCCT
2578,2023-01-05 18:50:18.370,46727,NaT,2026-01-21,2025-01-15,NaT,Não se Aplica,Válido,Não se Aplica,Válido
227,2021-10-01 11:01:42.170,35815,NaT,2025-09-10,2025-09-10,2025-09-10,Válido,Não se Aplica,Não se Aplica,Não se Aplica
1321,2023-01-03 00:44:17.907,49470,NaT,2030-08-25,2030-08-25,2030-08-25,Não se Aplica,Válido,Não se Aplica,Válido
464,2022-10-05 20:50:20.227,40866,2018-10-10,2032-03-03,2032-03-03,2032-03-03,Não se Aplica,Válido,Não se Aplica,Válido
720,2023-01-03 14:02:05.113,47477,NaT,2031-07-16,2031-07-16,2031-07-16,Não se Aplica,Válido,Não se Aplica,Não Assinado


In [77]:
definicoes =   input_rapeel #pd.merge(input_rapeel,ug_rapeel,on=ide_data_cols,how='inner')
dict_validades = {
    'DatValidadeLO' : 'LO',
    'DatValidadeLI' : 'LI',
    'DatValidadeLP' : "LP"
}

definicoes['ValidadeAmbiental'] = pd.NA
for validade in dict_validades:
    definicoes.loc[definicoes.ValidadeAmbiental.isna(),'ValidadeAmbiental'] = definicoes[validade]
definicoes[['IdeUsinaOutorga'] +list(dict_validades.keys()) + ['ValidadeAmbiental']].sample(5)

,IdeUsinaOutorga,DatValidadeLO,DatValidadeLI,DatValidadeLP,ValidadeAmbiental
963,49821,NaT,NaT,2022-12-30,2022-12-30
635,31271,NaT,2020-09-19,2013-01-18,2020-09-19
1026,46567,2027-04-25,2027-04-25,2027-04-25,2027-04-25
1138,49688,NaT,2024-06-29,2023-02-22,2024-06-29
1381,54778,NaT,NaT,2024-11-19,2024-11-19


In [78]:
cols = ["IdeUsinaOutorga","DatCanteiroObraRealizado","DatDesvioRioRealizado","DatEnchimentoRealizado","DatConclusaoSisTransRealizado","DatPrevisaoIniciobra","IdcObraParalisada","IdcUsinaMonitorada","DatMontagemOutorgado","DatConcretagemRealizado","DatConclusaoTorresRealizado","DatInicioObraOutorgado","DatConcretagemOutorgado","DatSisTransmissaoRealizado","IdcSituacaoObra","IdcSemPrevisao","DatComissionamentoUGRealizado","DatMonitoramento","SigTipoGeracao","DscComercializacaoEnergia","DatInicioObraRealizado","DatMontagemRealizado"]

monitoramentodados = pd.read_parquet(os.path.join(directory,"vmonitoramentousina.gzip"))[cols]

In [79]:
#WORK.dadosAcesso t1.'DthEnvio' LABEL="datadainformacao" AS datadainformacao
rename_cols = {'DatPrevistaComercial':'DatPrevistaComercial'}
definicoes_monitoramento = pd.merge(monitoramentodados,definicoes,on='IdeUsinaOutorga',how="left").rename(columns=rename_cols).drop_duplicates()
definicoes_monitoramento = definicoes_monitoramento.loc[definicoes_monitoramento.IdcUsinaMonitorada == "Sim"]

definicoes_monitoramento['CondicaoAmbiental'] = pd.NA

for validade in reversed(dict_validades):
    definicoes_monitoramento.loc[definicoes_monitoramento[validade].notna(),'CondicaoAmbiental'] = dict_validades[validade]

definicoes_monitoramento.loc[(definicoes_monitoramento.CondicaoAmbiental.isna()) & (definicoes_monitoramento.DthEnvio.notna()),'CondicaoAmbiental'] = 'Sem LP'
definicoes_monitoramento.loc[(definicoes_monitoramento.CondicaoAmbiental.isna()),'CondicaoAmbiental'] = 'Não Informado'


definicoes_monitoramento[list(dict_validades) + ['ValidadeAmbiental','CondicaoAmbiental']].sample(10)

display(definicoes_monitoramento[list(dict_validades.keys()) + ['CondicaoAmbiental','ValidadeAmbiental']])

definicoes_monitoramento['CondicaoConexao'] = "Não informado"

nsa = "Não se Aplica"
nass = "Não Assinado"
valido = "Válido"
vencido = "Vencido"

definicoes_monitoramento.loc[
    ((definicoes_monitoramento.NomSitContratoCCD ==  nsa)&
    (definicoes_monitoramento.NomSitContratoCCT == nsa) &
    definicoes_monitoramento.DthEnvio.notna()
    )
    ,'CondicaoConexao'] = "Verificar"

definicoes_monitoramento.loc[
(    ((definicoes_monitoramento.NomSitContratoCCD ==  valido) |
    (definicoes_monitoramento.NomSitContratoCCT == valido)) &
    definicoes_monitoramento.DthEnvio.notna())
    
    ,'CondicaoConexao'] = "OK"

definicoes_monitoramento.loc[
    (((definicoes_monitoramento.NomSitContratoCCD ==  vencido) |
    (definicoes_monitoramento.NomSitContratoCCT == vencido) |
    (definicoes_monitoramento.NomSitContratoCCD == nass) |
    (definicoes_monitoramento.NomSitContratoCCT == nass )) &
    definicoes_monitoramento.DthEnvio.notna())
    ,'CondicaoConexao'] = "Sem Conexão"
display(definicoes_monitoramento[['IdeUsinaOutorga'] + ['NomSitContratoCCD','NomSitContratoCCT','DthEnvio','CondicaoConexao']].sample(10))


definicoes_monitoramento['CondicaoUso'] = "Não informado"

definicoes_monitoramento.loc[
    ((definicoes_monitoramento.NomSituacaoContratoCUSD ==  nsa)&
    (definicoes_monitoramento.NomSitContratoCUST == nsa) 
    )
    ,'CondicaoUso'] = "Verificar"


definicoes_monitoramento.loc[
    ((definicoes_monitoramento.NomSituacaoContratoCUSD ==  valido) |
    (definicoes_monitoramento.NomSitContratoCUST == valido) 
    )
    ,'CondicaoUso'] = "OK"

definicoes_monitoramento.loc[
    (((definicoes_monitoramento.NomSituacaoContratoCUSD ==  vencido) |
    (definicoes_monitoramento.NomSitContratoCUST == vencido) |
    (definicoes_monitoramento.NomSituacaoContratoCUSD == nass) |
    (definicoes_monitoramento.NomSitContratoCUST == nass )))
    ,'CondicaoUso'] = "Sem Uso"

display(definicoes_monitoramento[['IdeUsinaOutorga'] + ['NomSituacaoContratoCUSD','NomSitContratoCUST','CondicaoConexao']].sample(10))


definicoes_monitoramento["PPA"] = "Ambos"

definicoes_monitoramento.loc[
    (((definicoes_monitoramento.DscComercializacaoEnergia == "ACR") & 
        definicoes_monitoramento.DatConclusaoACL.isna())),
    "PPA"] = "ACR"

definicoes_monitoramento.loc[
    (((definicoes_monitoramento.DscComercializacaoEnergia == "Fora do ACR") & 
        definicoes_monitoramento.DatConclusaoACL.isna())),
    "PPA"] = "Nenhum"

definicoes_monitoramento.loc[
    (((definicoes_monitoramento.DscComercializacaoEnergia == "Fora do ACR") & 
        definicoes_monitoramento.DatConclusaoACL.notna())),
    "PPA"] = "ACL"

display(definicoes_monitoramento[['IdeUsinaOutorga'] + ['DscComercializacaoEnergia','DatConclusaoACL','PPA']].sample(10))

,DatValidadeLO,DatValidadeLI,DatValidadeLP,CondicaoAmbiental,ValidadeAmbiental
1,NaT,2026-03-09,2002-12-11,LI,2026-03-09
2,NaT,NaT,NaT,Sem LP,NaT
3,2026-02-22,2028-06-28,2000-03-29,LO,2026-02-22
7,NaT,NaT,NaT,Não Informado,NaT
8,NaT,2009-03-10,1999-01-09,LI,2009-03-10
...,...,...,...,...,...
3093,NaT,NaT,NaT,Não Informado,NaT
3094,NaT,NaT,NaT,Não Informado,NaT
3095,NaT,NaT,NaT,Não Informado,NaT
3096,NaT,NaT,NaT,Não Informado,NaT


,IdeUsinaOutorga,NomSitContratoCCD,NomSitContratoCCT,DthEnvio,CondicaoConexao
2247,49654,Não Assinado,Não se Aplica,2022-12-05 14:44:03.383,Sem Conexão
1482,45026,Válido,Não se Aplica,2023-01-05 22:24:00.727,OK
1369,44448,Não se Aplica,Não Assinado,2023-01-02 15:30:03.197,Sem Conexão
2962,54904,Não se Aplica,Não se Aplica,2023-01-03 16:56:15.217,Verificar
842,37632,Não se Aplica,Válido,2023-01-05 15:42:30.587,OK
813,37566,Não Assinado,Não Assinado,2023-01-05 19:39:23.077,Sem Conexão
1601,46568,Não se Aplica,Válido,2023-01-02 12:12:27.180,OK
2634,51690,Não Assinado,Não Assinado,2023-01-14 23:08:14.703,Sem Conexão
171,31809,Não se Aplica,Não se Aplica,2023-01-03 16:35:20.787,Verificar
2639,51695,NaN,NaN,NaT,Não informado


,IdeUsinaOutorga,NomSituacaoContratoCUSD,NomSitContratoCUST,CondicaoConexao
1400,44539,Não se Aplica,Válido,Sem Conexão
2780,53696,Não se Aplica,Válido,Sem Conexão
1998,48762,Não se Aplica,Não Assinado,Sem Conexão
3044,56665,Não se Aplica,Não Assinado,Sem Conexão
2982,55335,NaN,NaN,Não informado
3026,56419,Não se Aplica,Não Assinado,Sem Conexão
1313,43275,Não se Aplica,Válido,Sem Conexão
2634,51690,Não Assinado,Não Assinado,Sem Conexão
2130,49418,Não se Aplica,Válido,OK
2983,55336,NaN,NaN,Não informado


,IdeUsinaOutorga,DscComercializacaoEnergia,DatConclusaoACL,PPA
2209,49516,Fora do ACR,NaT,Nenhum
1848,47398,Fora do ACR,NaT,Nenhum
790,37494,Fora do ACR,2021-05-03,ACL
1821,47367,Fora do ACR,NaT,Nenhum
927,37771,Fora do ACR,2022-07-05,ACL
1797,47339,Fora do ACR,NaT,Nenhum
2713,52257,Fora do ACR,NaT,Nenhum
1769,47301,Fora do ACR,NaT,Nenhum
1110,38352,Fora do ACR,2021-07-15,ACL
2315,49840,Fora do ACR,NaT,Nenhum


In [80]:
# Apenas mudança de nome
tabela_criterio = definicoes_monitoramento

In [81]:
ea = "Em andamento"
na = "Não Iniciada"
nenhum = "Nenhum"
paralisada = "Paralisada"
ok = "OK"

tabela_criterio['criterio'] = np.select(
    [
        (tabela_criterio.IdcSituacaoObra == ea),
        (( tabela_criterio.IdcSituacaoObra == na)  &  (tabela_criterio.PPA != nenhum) ),
        ((tabela_criterio.IdcSituacaoObra == na) & (tabela_criterio.PPA == nenhum) & (tabela_criterio.CondicaoAmbiental.isin(["LI","LO"])) & (tabela_criterio.CondicaoUso == ok)),
        ((tabela_criterio.IdcSituacaoObra == na) & (tabela_criterio.PPA == nenhum) & (tabela_criterio.CondicaoAmbiental.isin(["LI","LO"])) & (tabela_criterio.CondicaoUso != ok)),
        ((tabela_criterio.IdcSituacaoObra == na) & (tabela_criterio.PPA == nenhum) & (tabela_criterio.CondicaoAmbiental == "LP") & (tabela_criterio.CondicaoUso == ok)),
        ((tabela_criterio.IdcSituacaoObra == na) & (tabela_criterio.PPA == nenhum) & (tabela_criterio.CondicaoAmbiental == "LP") & (tabela_criterio.CondicaoUso != ok)),
        (tabela_criterio.IdcSituacaoObra == paralisada),
        ((tabela_criterio.DthEnvio.isna()) & (tabela_criterio.PPA == nenhum))
    ],
    [0,1,2,3,4,5,8,7],
    default=6
)


tabela_criterio['dscriterio'] = np.select(
    [
        (tabela_criterio.IdcSituacaoObra == ea),
        (( tabela_criterio.IdcSituacaoObra == na)  &  (tabela_criterio.PPA != nenhum) ),
        ((tabela_criterio.IdcSituacaoObra == na) & (tabela_criterio.PPA == nenhum) & (tabela_criterio.CondicaoAmbiental.isin(["LI","LO"])) & (tabela_criterio.CondicaoUso == ok)),
        ((tabela_criterio.IdcSituacaoObra == na) & (tabela_criterio.PPA == nenhum) & (tabela_criterio.CondicaoAmbiental.isin(["LI","LO"])) & (tabela_criterio.CondicaoUso != ok)),
        ((tabela_criterio.IdcSituacaoObra == na) & (tabela_criterio.PPA == nenhum) & (tabela_criterio.CondicaoAmbiental == "LP") & (tabela_criterio.CondicaoUso == ok)),
        ((tabela_criterio.IdcSituacaoObra == na) & (tabela_criterio.PPA == nenhum) & (tabela_criterio.CondicaoAmbiental == "LP") & (tabela_criterio.CondicaoUso != ok)),
        (tabela_criterio.IdcSituacaoObra == paralisada),
        ((tabela_criterio.DthEnvio.isna()) & (tabela_criterio.PPA == nenhum))
    ],
    [
    "Usina em Obras em andamento",
    "Sem obras e PPA Ok",
    "Sem obras, sem PPA e LI Ok e Cust OK",
    "Sem obras, sem PPA e LI OK e Cust não Ok",
    "Sem obras, sem PPA e LP OK e Cust Ok",
    "Sem obras, sem PPA e LP OK e Cust não Ok",
    "Obras paralisadas",
    "Sem obras, sem PPA e sem Rapeel"
],
    default="Sem LP")

display(tabela_criterio[['IdeUsinaOutorga','IdcSituacaoObra','PPA','CondicaoAmbiental','CondicaoUso','criterio','dscriterio']].sample(10))

,IdeUsinaOutorga,IdcSituacaoObra,PPA,CondicaoAmbiental,CondicaoUso,criterio,dscriterio
1875,47448,Não Iniciada,Nenhum,LO,OK,2,"Sem obras, sem PPA e LI Ok e Cust OK"
1897,47470,Não Iniciada,Nenhum,LO,OK,2,"Sem obras, sem PPA e LI Ok e Cust OK"
3073,70538,Não Iniciada,Nenhum,Não Informado,Não informado,7,"Sem obras, sem PPA e sem Rapeel"
2929,54715,Não Iniciada,Nenhum,LI,Sem Uso,3,"Sem obras, sem PPA e LI OK e Cust não Ok"
2445,50468,Em andamento,ACL,LO,OK,0,Usina em Obras em andamento
3036,56429,Não Iniciada,Nenhum,LO,Sem Uso,3,"Sem obras, sem PPA e LI OK e Cust não Ok"
2195,49502,Não Iniciada,Nenhum,LO,OK,2,"Sem obras, sem PPA e LI Ok e Cust OK"
263,32584,Em andamento,Nenhum,LO,OK,0,Usina em Obras em andamento
2930,54716,Não Iniciada,Nenhum,LI,Sem Uso,3,"Sem obras, sem PPA e LI OK e Cust não Ok"
2842,54126,Não Iniciada,Nenhum,LI,Sem Uso,3,"Sem obras, sem PPA e LI OK e Cust não Ok"


In [82]:
slp = "Sem LP"
sim = "Sim"
tabela_criterio['criterio_novo'] = np.select(
    [
        (tabela_criterio.IdcSemPrevisao == sim),
        (tabela_criterio.IdcSituacaoObra == ea),
        (( tabela_criterio.IdcSituacaoObra == na)  &  (tabela_criterio.PPA != nenhum) ),
        ((tabela_criterio.IdcSituacaoObra == na) & (tabela_criterio.PPA == nenhum) & (tabela_criterio.CondicaoAmbiental.isin(["LI","LO"])) & (tabela_criterio.CondicaoUso == ok)),
        ((tabela_criterio.IdcSituacaoObra == na) & (tabela_criterio.PPA == nenhum) & (tabela_criterio.CondicaoAmbiental == "LP") & (tabela_criterio.CondicaoUso == ok)),
        ((tabela_criterio.IdcSituacaoObra == na) & (tabela_criterio.PPA == nenhum) & (tabela_criterio.CondicaoAmbiental.isin(["LI","LO"])) & (tabela_criterio.CondicaoUso != ok)),
        ((tabela_criterio.IdcSituacaoObra == na) & (tabela_criterio.PPA == nenhum) & (tabela_criterio.CondicaoAmbiental == "LP") & (tabela_criterio.CondicaoUso != ok)),
        (tabela_criterio.IdcSituacaoObra == paralisada),
        (tabela_criterio.CondicaoAmbiental == "Sem LP"),
        ((tabela_criterio.DthEnvio.isna()) & (tabela_criterio.PPA == nenhum)),
        
    ],
    [9,0,1,2,4,3,5,8,6,7],
    default=-8
)


display(tabela_criterio[['IdeUsinaOutorga','IdcSituacaoObra','PPA','CondicaoAmbiental','CondicaoUso','criterio_novo','dscriterio']].sample(5))

,IdeUsinaOutorga,IdcSituacaoObra,PPA,CondicaoAmbiental,CondicaoUso,criterio_novo,dscriterio
2865,54336,Não Iniciada,Nenhum,LP,Sem Uso,5,"Sem obras, sem PPA e LP OK e Cust não Ok"
2798,53828,Não Iniciada,Nenhum,LI,Sem Uso,3,"Sem obras, sem PPA e LI OK e Cust não Ok"
2888,54624,Não Iniciada,Nenhum,LO,Sem Uso,3,"Sem obras, sem PPA e LI OK e Cust não Ok"
1465,44987,Não Iniciada,Nenhum,LP,Sem Uso,5,"Sem obras, sem PPA e LP OK e Cust não Ok"
2808,53841,Não Iniciada,Nenhum,LP,Sem Uso,5,"Sem obras, sem PPA e LP OK e Cust não Ok"


In [83]:
previsao_path = os.path.join(get_standard_folder_path("Documents"),r"Previsor\Previsoes")
previsao_file = sorted(glob.glob(f"{previsao_path}/*.gzip"))[-1]
calculo_previsao = pd.read_parquet(previsao_file)
calculo_previsao.FaseAtual =calculo_previsao.FaseAtual.str.slice(0,-3)
calculo_previsao.Indicador =calculo_previsao.Indicador/100


In [84]:
cols = ["Dat_OC_obrigacao","DatPrevisaoIniciobra","FASE","Ind_crono_norm","IdeUsinaOutorga","flagOPTeste30dias",'Previsao_OC']
rename_cols = {'FaseAtual':'FASE','Indicador':'Ind_crono_norm'}
calculo_previsao = calculo_previsao.rename(columns=rename_cols)

for col in ['SigTipoGeracao','DatMonitoramento ']:
    if 'SigTipoGeracao' in calculo_previsao.columns:
        calculo_previsao.drop(['SigTipoGeracao'],inplace=True,axis=1)

### criterios_da_previsao

In [85]:
criterios_da_previsao  = pd.merge(
                                    tabela_criterio,calculo_previsao,
                                    on="IdeUsinaOutorga",how='inner'
                                    )
criterios_da_previsao = pd.merge(criterios_da_previsao,ug_rapeel,left_on= ['IdeUsinaOutorga','NumUgUsina'],right_on=['IdeUsinaOutorga','NumOperacaoUg'],how='left')

del criterios_da_previsao['NumOperacaoUg']

# Regras

In [86]:
"""################################ regra_antiga ################################
criterios_da_previsao.loc[(criterios_da_previsao['IdcSemPrevisao'] == "Sim"),'regra_antiga'] = pd.NA
criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] != "OT"),'regra_antiga'] = criterios_da_previsao[['Previsao_OC','Dat_OC_obrigacao']].max(axis=1)
criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] == "OT") & (criterios_da_previsao['flagOPTeste30dias'] == 1),'regra_antiga'] = ( hoje +pd.Timedelta(60,'D'))
criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] == "OT"),'regra_antiga'] = criterios_da_previsao['Previsao_OC']
criterios_da_previsao.loc[( (criterios_da_previsao['criterio'] == 1) | (criterios_da_previsao['criterio'] == 2)) & ( (criterios_da_previsao['IdcObraParalisada'] == "Sim")),'regra_antiga'] = criterios_da_previsao[['DatPrevistaComercial','Dat_OC_obrigacao']].max(axis=1)
criterios_da_previsao.loc[( (criterios_da_previsao['criterio'] == 1) | (criterios_da_previsao['criterio'] == 2)) & ( (criterios_da_previsao['IdcObraParalisada'] == "Não")),'regra_antiga'] = criterios_da_previsao[['DatPrevistaComercial','Previsao_OC']].max(axis=1)
criterios_da_previsao.loc[(criterios_da_previsao['criterio'] >= 3) & ( (criterios_da_previsao['IdcObraParalisada'] == "Sim")),'regra_antiga'] = criterios_da_previsao[['DatPrevistaComercial','Dat_OC_obrigacao']].max(axis=1)
criterios_da_previsao.loc[criterios_da_previsao['criterio'] >= 3 & ( (criterios_da_previsao['IdcObraParalisada'] == "Não")),'regra_antiga'] = criterios_da_previsao[['DatPrevistaComercial','Previsao_OC','Dat_OC_obrigacao']].max(axis=1)


################################ regrapmo ################################
criterios_da_previsao.loc[(criterios_da_previsao['IdcSemPrevisao'] == "Sim"),'regrapmo'] =  pd.NA
criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] != "OT"),'regrapmo'] = criterios_da_previsao[['DatPrevistaComercial','Previsao_OC']].max(axis=1)
criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] == "OT") & (criterios_da_previsao['flagOPTeste30dias'] == 1),'regrapmo'] = ( hoje + pd.Timedelta(60,'D'))
criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] == "OT"),'regrapmo'] = criterios_da_previsao['Previsao_OC']
criterios_da_previsao.loc[( (criterios_da_previsao['criterio'] == 8) ),'regrapmo'] = criterios_da_previsao[['DatPrevistaComercial','Dat_OC_obrigacao']].max(axis=1)
criterios_da_previsao.loc[( (criterios_da_previsao['criterio'] == 1) | (criterios_da_previsao['criterio'] == 2)) & ( (criterios_da_previsao['IdcObraParalisada'] == "Não")),'regrapmo'] = criterios_da_previsao[['DatPrevistaComercial','Dat_OC_obrigacao','Previsao_OC']].max(axis=1)

criterios_da_previsao["Dummy"] = hoje + pd.Timedelta(3*365,'D')
criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 3),'regrapmo'] = criterios_da_previsao[['DatPrevistaComercial','Dat_OC_obrigacao','Previsao_OC','Dummy']].max(axis=1)

criterios_da_previsao["Dummy"] = hoje + pd.Timedelta(4*365,'D')
criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 4),'regrapmo'] = criterios_da_previsao[['DatPrevistaComercial','Dat_OC_obrigacao','Previsao_OC','Dummy']].max(axis=1)

criterios_da_previsao["Dummy"] = hoje + pd.Timedelta(5*365,'D')
criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 5) | (criterios_da_previsao['criterio'] == 7),'regrapmo'] =  criterios_da_previsao[['DatPrevistaComercial','Dat_OC_obrigacao','Previsao_OC','Dummy']].max(axis=1)

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 6),'regrapmo'] = pd.NA"""

'################################ regra_antiga ################################\ncriterios_da_previsao.loc[(criterios_da_previsao[\'IdcSemPrevisao\'] == "Sim"),\'regra_antiga\'] = pd.NA\ncriterios_da_previsao.loc[(criterios_da_previsao[\'criterio\'] == 0) & (criterios_da_previsao[\'FASE\'] != "OT"),\'regra_antiga\'] = criterios_da_previsao[[\'Previsao_OC\',\'Dat_OC_obrigacao\']].max(axis=1)\ncriterios_da_previsao.loc[(criterios_da_previsao[\'criterio\'] == 0) & (criterios_da_previsao[\'FASE\'] == "OT") & (criterios_da_previsao[\'flagOPTeste30dias\'] == 1),\'regra_antiga\'] = ( hoje +pd.Timedelta(60,\'D\'))\ncriterios_da_previsao.loc[(criterios_da_previsao[\'criterio\'] == 0) & (criterios_da_previsao[\'FASE\'] == "OT"),\'regra_antiga\'] = criterios_da_previsao[\'Previsao_OC\']\ncriterios_da_previsao.loc[( (criterios_da_previsao[\'criterio\'] == 1) | (criterios_da_previsao[\'criterio\'] == 2)) & ( (criterios_da_previsao[\'IdcObraParalisada\'] == "Sim")),\'regra_antiga\'] = criterios_da_p

In [87]:
"""################################ dscjustificativapmo ################################
list_condicoes = [
    (criterios_da_previsao['IdcSemPrevisao'] == "Sim"),
    (criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] != "OT"), #& ( criterios_da_previsao['Previsao_OC'] > criterios_da_previsao['DatPrevistaComercial']),
    (criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] != "OT"), #& ( criterios_da_previsao['Previsao_OC'] < criterios_da_previsao['DatPrevistaComercial']),
    (criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] == "OT"),
    (criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] == "OT") & (criterios_da_previsao['flagOPTeste30dias'] == 1),
    ( (criterios_da_previsao['criterio'] == 8)) & ( criterios_da_previsao['DatPrevistaComercial'] > criterios_da_previsao['Dat_OC_obrigacao']),
    ( (criterios_da_previsao['criterio'] == 8)) & ( criterios_da_previsao['DatPrevistaComercial'] < criterios_da_previsao['Dat_OC_obrigacao']),
    ( (criterios_da_previsao['criterio'] == 1) | (criterios_da_previsao['criterio'] == 2)) & ( (criterios_da_previsao['IdcObraParalisada'] == "Não")), # & ( criterios_da_previsao['DatPrevistaComercial'] > criterios_da_previsao['Previsao_OC']),
    ( (criterios_da_previsao['criterio'] == 1) | (criterios_da_previsao['criterio'] == 2)) & ( (criterios_da_previsao['IdcObraParalisada'] == "Não")), # & ( criterios_da_previsao['DatPrevistaComercial'] < criterios_da_previsao['Previsao_OC']), 
    (criterios_da_previsao['criterio'] == 3),
    (criterios_da_previsao['criterio'] == 4) | (criterios_da_previsao['criterio'] == 5),
    (criterios_da_previsao['criterio'] == 7),
    (criterios_da_previsao['criterio'] == 6),
]

list_values = [
    "Analisar justificativa",
    "Estágio Atual das Obras",
    "Estágio Atual das Obras",
    "Usina liberada para testes",
    "Testes não concluídos",
    "Paralisação de Obras",
    "Paralisação de Obras",
    "Estágio Atual das Obras",
    "Estágio Atual das Obras" ,
    "Não celebrado contrato de uso do sistema de transmissão/distribuição",
    "Situação da Licença de Instalação",
    "Sem Rapeel",
    "Sem Licenciamento Ambiental"
]

criterios_da_previsao['dscjustificativapmo'] = np.select(list_condicoes,list_values,default="Null")"""


################################ criterionovopmo ################################
list_condicoes = [
    ((criterios_da_previsao['criterio_novo'] == 0) & (criterios_da_previsao['FASE'] != "OT")),
    ((criterios_da_previsao['criterio_novo'] == 0) & (criterios_da_previsao['FASE'] == "OT") & (criterios_da_previsao['flagOPTeste30dias'] == 1)),
    ((criterios_da_previsao['criterio_novo'] == 0) & (criterios_da_previsao['FASE'] == "OT")),
    ((criterios_da_previsao['criterio_novo'] == 1)),
    ((criterios_da_previsao['criterio_novo'] == 2)),
    ((criterios_da_previsao['criterio_novo'] == 3)),
    ((criterios_da_previsao['criterio_novo'] == 5)),
    ((criterios_da_previsao['criterio_novo'] == 4)),
    ((criterios_da_previsao['criterio_novo'] == 6)),
    ((criterios_da_previsao['criterio_novo'] == 7)),
    ((criterios_da_previsao['criterio_novo'] == 8)),
    ((criterios_da_previsao['criterio_novo'] == 9)),
]

list_values = [0.1, 0.2, 0.3, 1, 2, 3, 5, 4, 6, 7, 8, 9]

criterios_da_previsao['criterionovopmo'] = np.select(list_condicoes,list_values)



list_condicoes = [
    ((criterios_da_previsao['criterio_novo'] == 9)),
    ((criterios_da_previsao['criterio_novo'] == 0) & (criterios_da_previsao['FASE'] != "OT")),
    ((criterios_da_previsao['criterio_novo'] == 0) & (criterios_da_previsao['FASE'] == "OT") & (criterios_da_previsao['flagOPTeste30dias'] == 1)),
    ((criterios_da_previsao['criterio_novo'] == 0) & (criterios_da_previsao['FASE'] == "OT")),
    ((criterios_da_previsao['criterio_novo'] == 1)),
    ((criterios_da_previsao['criterio_novo'] == 2)),
    ((criterios_da_previsao['criterio_novo'] == 3)),
    ((criterios_da_previsao['criterio_novo'] == 4)),
    ((criterios_da_previsao['criterio_novo'] == 5)),
    ((criterios_da_previsao['criterio_novo'] == 7)),
    ((criterios_da_previsao['criterio_novo'] == 6)),
    ((criterios_da_previsao['criterio_novo'] == 8))
]


list_values = [
    "Usina Viabilidade Baixa análise da fiscalização",
    "Usina em obras == Previsão OC maior entre data calculada e data RAPEEL",
    "Usina em Teste há mais de 30 dias = Previsão OC próximos 60 dias",
    "Usina em Teste = Previsão OC conforme data calculada",
    "Usina sem obras com PPA = Previsão OC maior entre data calculada, data RAPEEL e data compromisso",
    "Usina sem obras com CUST = Previsão OC maior entre data calculada, data RAPEEL e data compromisso",
    "Usina sem obras, sem cust, sem PPA e com LI = Previsão OC handicap de 5 anos",
    "Usina sem obras, com cust, sem PPA e sem LI = Previsão OC handicap de 5 anos",
    "Usina sem obras, sem cust, sem PPA e sem LI = Previsão OC handicap de 5 anos",
    "Usina sem obras, sem PPA e sem RAPEEL = Previsão OC handicap de 5 anos",
    "Usina sem LP = Previsão OC sem previsão","Usina obras paralisadas == Previsão OC maior entre data RAPEEL e data compromisso"
]


criterios_da_previsao['dsccriterionovo'] = np.select(list_condicoes,list_values)


In [88]:
################################ regranovapmo ################################
criterios_da_previsao.loc[(criterios_da_previsao['IdcSemPrevisao'] == "Sim"),'regranovapmo'] = pd.NA

criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo'] == 0) & (criterios_da_previsao['FASE'] != "OT"),'regranovapmo'] = criterios_da_previsao[['DatPrevistaComercial','Previsao_OC']].max(axis=1)

criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo'] == 0) & (criterios_da_previsao['FASE'] == "OT") & (criterios_da_previsao['flagOPTeste30dias'] == 1),'regranovapmo'] = ( hoje + pd.Timedelta(60,"D"))

criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo'] == 0) & (criterios_da_previsao['FASE'] == "OT") & (criterios_da_previsao['Previsao_OC'].isna()),'regranovapmo'] = criterios_da_previsao['DatPrevistaComercial']

criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo'] == 0) & (criterios_da_previsao['FASE'] == "OT") & (criterios_da_previsao['Previsao_OC'].notna()),'regranovapmo'] = criterios_da_previsao['Previsao_OC']

criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo']== 1) | (criterios_da_previsao['criterio_novo']== 2) ,'regranovapmo'] = criterios_da_previsao[['DatPrevistaComercial','Previsao_OC','Dat_OC_obrigacao']].max(axis=1)
criterios_da_previsao["Dummy"] = hoje + pd.Timedelta(5*365,'D')

criterios_da_previsao.loc[((criterios_da_previsao['criterio_novo'] == 3) | (criterios_da_previsao['criterio_novo'] == 4) | (criterios_da_previsao['criterio_novo'] == 5) ),'regranovapmo'] = criterios_da_previsao[['DatPrevistaComercial','Previsao_OC','Dat_OC_obrigacao','Dummy']].max(axis=1)

criterios_da_previsao.loc[criterios_da_previsao['criterio_novo']== 6,'regranovapmo'] = pd.NA

criterios_da_previsao["Dummy"] = hoje + pd.Timedelta(6*365,'D')
criterios_da_previsao.loc[criterios_da_previsao['criterio_novo']== 7,'regranovapmo'] =  criterios_da_previsao[['DatPrevistaComercial','Previsao_OC','Dat_OC_obrigacao','Dummy']].max(axis=1)

criterios_da_previsao.loc[criterios_da_previsao['criterio_novo']== 8,'regranovapmo'] = criterios_da_previsao[['DatPrevistaComercial','Dat_OC_obrigacao']].max(axis=1)


################################ criterio_pmo_ts ################################
list_condicoes = [
    (criterios_da_previsao['IdcSemPrevisao'] == "Sim"),
    (criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] != "OT"),
    (criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] == "OT") & (criterios_da_previsao['flagOPTeste30dias'] == 1),
    (criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] == "OT"),
    ((criterios_da_previsao['PPA'] != "Nenhum")) & ((criterios_da_previsao['criterio'] == 8) ),
    ( (criterios_da_previsao['criterio'] == 1)) & ( (criterios_da_previsao['IdcObraParalisada'] == "Não")),
    ((criterios_da_previsao['criterio'] == 2)) & ( (criterios_da_previsao['IdcObraParalisada'] == "Não")),
    ( (criterios_da_previsao['criterio'] == 8)) & ( (criterios_da_previsao['PPA'] == "Nenhum") & (criterios_da_previsao['CondicaoUso'] == "OK")) & ( (criterios_da_previsao['CondicaoAmbiental'] == "LI") | (criterios_da_previsao['CondicaoAmbiental'] == "LO") ),
    (criterios_da_previsao['criterio'] == 3),
    (criterios_da_previsao['criterio'] == 4),
    (criterios_da_previsao['criterio'] == 5),
    (criterios_da_previsao['criterio'] == 7),
    (criterios_da_previsao['criterio'] == 6),
]

list_values = [8,0.1,0.2,0.3,1.1,1.2,2.2,2.1,3,4,5,7,6]
criterios_da_previsao['criterio_pmo_ts'] = np.select(list_condicoes,list_values)


In [89]:
################################ dsccriterio_pmo_ts ################################
criterios_da_previsao.loc[(criterios_da_previsao['IdcSemPrevisao'] == "Sim"),'dsccriterio_pmo_ts'] = "Usina Viabilidade Baixa análise da fiscalização"

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] != "OT"),'dsccriterio_pmo_ts'] = "Usina em obras == Previsão OC maior entre data calculada e data RAPEEL"

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] == "OT") & (criterios_da_previsao['flagOPTeste30dias'] == 1),'dsccriterio_pmo_ts'] = "Usina em Teste há mais de 30 dias == Previsão OC próximos 60 dias"

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] == "OT"),'dsccriterio_pmo_ts'] = "Usina em Teste == Previsão OC conforme data calculada"

criterios_da_previsao.loc[((criterios_da_previsao['PPA'] != "Nenhum")) & ((criterios_da_previsao['criterio'] == 8) ),'dsccriterio_pmo_ts'] = "Usina obras paralisadas com PPA = Previsão OC maior entre data RAPEEL e data compromisso"

criterios_da_previsao.loc[( (criterios_da_previsao['criterio'] == 1)) & ( (criterios_da_previsao['IdcObraParalisada'] == "Não")),'dsccriterio_pmo_ts'] = "Usina sem obras com PPA = Previsão OC maior entre data calculada, data RAPEEL e data compromisso"

criterios_da_previsao.loc[(criterios_da_previsao['IdcSemPrevisao'] == "Sim"),'dsccriterio_pmo_ts'] = "Usina Viabilidade Baixa análise da fiscalização"

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] != "OT"),'dsccriterio_pmo_ts'] = "Usina em obras == Previsão OC maior entre data calculada e data RAPEEL"

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] == "OT") & (criterios_da_previsao['flagOPTeste30dias'] == 1),'dsccriterio_pmo_ts'] = "Usina em Teste há mais de 30 dias = Previsão OC próximos 60 dias"

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] == "OT"),'dsccriterio_pmo_ts'] = "Usina em Teste = Previsão OC conforme data calculada"

criterios_da_previsao.loc[((criterios_da_previsao['PPA'] != "Nenhum")) & ((criterios_da_previsao['criterio'] == 8) ),'dsccriterio_pmo_ts'] = "Usina obras paralisadas com PPA = Previsão OC maior entre data RAPEEL e data compromisso"

criterios_da_previsao.loc[( (criterios_da_previsao['criterio'] == 1)) & ( (criterios_da_previsao['IdcObraParalisada'] == "Não")),'dsccriterio_pmo_ts'] = "Usina sem obras com PPA = Previsão OC maior entre data calculada, data RAPEEL e data compromisso"

criterios_da_previsao.loc[((criterios_da_previsao['criterio'] == 2)) & ( (criterios_da_previsao['IdcObraParalisada'] == "Não")),'dsccriterio_pmo_ts'] = "Usina sem obras sem PPA com LI com CUST = Previsão OC maior entre data calculada, data RAPEEL e data compromisso"

criterios_da_previsao.loc[( (criterios_da_previsao['criterio'] == 8)) & ( (criterios_da_previsao['PPA'] == "Nenhum") & (criterios_da_previsao['CondicaoUso'] == "OK")) & ( (criterios_da_previsao['CondicaoAmbiental'] == "LI") | (criterios_da_previsao['CondicaoAmbiental'] == "LO") ),'dsccriterio_pmo_ts'] = "Usina obras paralisadas sem PPA com LI com CUST = Previsão OC maior entre data calculada, data RAPEEL e data compromisso"

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 3),'dsccriterio_pmo_ts'] = "Usina sem PPA com LI sem CUST = Previsão OC maior entre handicap de 3 anos e data RAPEEL e data compromisso e data calculada"

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 4),'dsccriterio_pmo_ts'] = "Usina sem PPA sem LI com CUST = Previsão OC maior entre handicap de 4 anos e data RAPEEL e data compromisso e data calculada"

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 5),'dsccriterio_pmo_ts'] = "Usina sem PPA sem LI sem CUST = Previsão OC maior entre handicap de 5 anos e data RAPEEL e data compromisso e data calculada"

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 7),'dsccriterio_pmo_ts'] = "Usina sem obras, sem PPA e sem RAPEEL = Previsão OC maior entre handicap de 5 anos e data compromisso e data calculada"

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 6),'dsccriterio_pmo_ts'] = "Usina sem LP = Previsão OC sem previsão"

In [90]:
################################ criterio_pmo_ts_max_usina ################################
criterio_pmo_ts_max = criterios_da_previsao.groupby("IdeUsinaOutorga")[['criterio_pmo_ts']].max().rename(columns=
{'criterio_pmo_ts': 'criterio_pmo_ts_max_usina'})
criterios_da_previsao = criterios_da_previsao.merge(criterio_pmo_ts_max,on="IdeUsinaOutorga",how='left')


################################ dscjustificativaregra ################################
criterios_da_previsao.loc[(criterios_da_previsao['IdcSemPrevisao'] == "Sim"),'dscjustificativaregra'] = "Analisar justificativa: Revogação da outorga em avaliação ou Demandas judiciais ou Inviabilidade da implantação da usina"

criterios_da_previsao.loc[(criterios_da_previsao['IdcObraParalisada'] == "Sim") & (criterios_da_previsao['IdcSemPrevisao'] != "Sim"),'dscjustificativaregra'] = "Paralisação de obras"

criterios_da_previsao.loc[(criterios_da_previsao.criterio_pmo_ts_max_usina== 0.2),'dscjustificativaregra'] = "Analisar justificativa: Sem Licença de Operação (LO) ou Alterações de características técnicas ou Estágio das obras da conexão associada ou Demandas judiciais ou Paralisação de obras"

criterios_da_previsao.loc[(criterios_da_previsao.criterio_pmo_ts_max_usina== 0.1) | (criterios_da_previsao['criterio_pmo_ts'] == 0.3),'dscjustificativaregra'] = "Estágio das obras da usina"


criterios_da_previsao.loc[(criterios_da_previsao.criterio_pmo_ts_max_usina== 1.1),'dscjustificativaregra'] = "Paralisação de obras"

criterios_da_previsao.loc[(criterios_da_previsao.criterio_pmo_ts_max_usina== 1.2),'dscjustificativaregra'] = "Compromisso de venda de energia - PPA"

criterios_da_previsao.loc[(criterios_da_previsao.criterio_pmo_ts_max_usina== 2.2),'dscjustificativaregra'] = "Licença de Instalação - LI e acesso a rede - CUST/CUSD"

criterios_da_previsao.loc[(criterios_da_previsao.criterio_pmo_ts_max_usina== 3),'dscjustificativaregra'] = "Acesso a rede contratado - CUST/CUSD"

criterios_da_previsao.loc[(criterios_da_previsao.criterio_pmo_ts_max_usina== 4),'dscjustificativaregra'] = "Acesso a rede não contratado - CUST/CUSD"

criterios_da_previsao.loc[(criterios_da_previsao.criterio_pmo_ts_max_usina== 5),'dscjustificativaregra'] = "Sem Licença de Instalação - LI"

criterios_da_previsao.loc[(criterios_da_previsao.criterio_pmo_ts_max_usina== 6),'dscjustificativaregra'] = "Nenhuma licença ambiental válida"

criterios_da_previsao.loc[(criterios_da_previsao.criterio_pmo_ts_max_usina== 7),'dscjustificativaregra'] = "Sem RAPEEL"


################################ dscjustificativaregranova ################################

list_condicoes = [
    (criterios_da_previsao['criterionovopmo'] == 9),
    (criterios_da_previsao['criterionovopmo'] == 8),
    (criterios_da_previsao['criterionovopmo'] == 0.2),
    (criterios_da_previsao['criterionovopmo'] == 0.1) | (criterios_da_previsao['criterio_pmo_ts'] == 0.3),
    (criterios_da_previsao['criterionovopmo'] == 1),
    criterios_da_previsao['criterionovopmo']==2,
    criterios_da_previsao['criterionovopmo']==3,
    (criterios_da_previsao.criterio_pmo_ts_max_usina== 4),
    (criterios_da_previsao.criterio_pmo_ts_max_usina== 5),
    (criterios_da_previsao.criterio_pmo_ts_max_usina== 6),
    (criterios_da_previsao.criterio_pmo_ts_max_usina== 7),
]

list_values = [
    "Analisar justificativa: Revogação da outorga em avaliação ou Demandas judiciais ou Inviabilidade da implantação da usina",
    "Paralisação de obras",
    "Analisar justificativa: Sem Licença de Operação (LO) ou Alterações de características técnicas ou Estágio das obras da conexão associada ou Demandas judiciais ou Paralisação de obras",
    "Estágio das obras da usina",
    "Compromisso de venda de energia - PPA",
    "Acesso a rede contratado - CUST/CUSD",
    "Acesso a rede não contratado - CUST/CUSD",
    "Sem Licença de Instalação - LI",
    "Sem Licença de Instalação - LI",
    "Nenhuma licença ambiental válida",
    "Sem RAPEEL"
]

criterios_da_previsao['dscjustificativaregranova'] = np.select(list_condicoes,list_values)


In [91]:
#criterios_da_previsao_sas = pd.read_excel("./WORK_Criteriosdaprevisao.xlsx")
#save_pickle(criterios_da_previsao_sas,'./criterios_da_previsao_sas.pickle')
#criterios_da_previsao_2_sas = pd.read_excel("./WORK_Criteriosdaprevisao2.xlsx")
#save_pickle(criterios_da_previsao_2_sas,'./criterios_da_previsao_2_sas.pickle')
#tabela_criterio_sas = pd.read_excel("./WORK_TABELCRITERIO.xlsx")
#save_pickle(tabela_criterio_sas,'./tabela_criterio_sas.pickle')

criterios_da_previsao_sas = load_pickle('./criterios_da_previsao_sas.pickle')
tabela_criterio_sas = load_pickle('./tabela_criterio_sas.pickle')
criterios_da_previsao_sas = pd.merge(criterios_da_previsao_sas,tabela_criterio_sas,on="IdeUsinaOutorga")


criterios_da_previsao_sas['SAS'] = True
criterios_da_previsao_sas.CalculoPrevisorOC_mx = criterios_da_previsao_sas.CalculoPrevisorOC_mx.dt.normalize()
criterios_da_previsao["PY"] = True

In [92]:
criterios_da_previsao_sas.DscJustificativaPMO = criterios_da_previsao_sas.DscJustificativaPMO.str.replace('_x0020_',' ')
criterios_da_previsao_sas.DscJustificativaPMO = criterios_da_previsao_sas.DscJustificativaPMO.str.replace('_x002F_','/')

In [93]:
criterios_da_previsao_sas.DscJustificativaRegranova = criterios_da_previsao_sas.DscJustificativaRegranova.str.replace('_x0020_',' ')
criterios_da_previsao_sas.DscJustificativaRegranova = criterios_da_previsao_sas.DscJustificativaRegranova.str.replace('_x002F_','/')
criterios_da_previsao_sas.DscJustificativaRegranova = criterios_da_previsao_sas.DscJustificativaRegranova.str.replace('_x0028_','(')
criterios_da_previsao_sas.DscJustificativaRegranova = criterios_da_previsao_sas.DscJustificativaRegranova.str.replace('_x0029_',')')

In [94]:
merge = pd.merge(criterios_da_previsao,criterios_da_previsao_sas,on=['IdeUsinaOutorga','NumUgUsina'],suffixes=('_py','_sas'),how='outer')
merge['OC_diff'] = merge.Previsao_OC - merge.CalculoPrevisorOC_mx

# Teste previsão

In [95]:
merge[['IdeUsinaOutorga','NumUgUsina'] + [ 'Previsao_OC','CalculoPrevisorOC_mx','OC_diff','IdcUsinaMonitorada']].sort_values(by='OC_diff',ascending=False).head(10)

,IdeUsinaOutorga,NumUgUsina,Previsao_OC,CalculoPrevisorOC_mx,OC_diff,IdcUsinaMonitorada
137,30414,1,2025-03-18,2022-09-08,922 days,Sim
143,30612,1,2025-03-29,2023-01-23,796 days,Sim
226,31691,4,2023-09-02,2022-06-09,450 days,Sim
228,31691,2,2023-09-02,2022-06-09,450 days,Sim
227,31691,7,2023-09-02,2022-06-09,450 days,Sim
221,31691,3,2023-09-02,2022-06-09,450 days,Sim
225,31691,1,2023-09-02,2022-06-09,450 days,Sim
223,31691,5,2023-09-02,2022-06-09,450 days,Sim
222,31691,6,2023-09-02,2022-06-09,450 days,Sim
224,31691,8,2023-09-02,2022-06-09,450 days,Sim


# Testes critérios

## FASE ok

In [96]:
merge[merge.FASE_py != merge.FASE_sas].shape

(1, 186)

## criterionovopmo OK

In [97]:
# Check
merge[merge.CriterioNovoPMO != merge.criterionovopmo][['IdeUsinaOutorga','NumUgUsina'] + ['IdcSemPrevisao','flagOPTeste30dias_sas','flagOPTeste30dias_py','FASE_sas','FASE_py','criterio_novo','criterio_sas','criterio_py','CriterioNovoPMO','criterionovopmo']]#.drop_duplicates(subset='IdeUsinaOutorga')

,IdeUsinaOutorga,NumUgUsina,IdcSemPrevisao,flagOPTeste30dias_sas,flagOPTeste30dias_py,FASE_sas,FASE_py,criterio_novo,criterio_sas,criterio_py,CriterioNovoPMO,criterionovopmo
74039,28060,1,NaN,0,NaN,ME,NaN,NaN,0,NaN,0.1,NaN


## criterio OK

In [98]:
# Check
merge[merge.criterio_sas != merge.criterio_py].shape

(1, 186)

## dscjustificativapmo

In [100]:
#merge[merge.DscJustificativaPMO  != merge.dscjustificativapmo][['IdeUsinaOutorga','NumUgUsina'] +['IdcSemPrevisao','IdcObraParalisada','FASE_sas','criterio_sas','DatPrevistaComercial','Previsao_OC','CalculoPrevisorOC_mx','DscJustificativaPMO','DatPrevistaComercial','dscjustificativapmo']]

## dscjustificativaregranova OK

In [ ]:
merge[merge.dscjustificativaregranova != merge.DscJustificativaRegranova]


,IdeUsinaOutorga,DatCanteiroObraRealizado,DatDesvioRioRealizado,DatEnchimentoRealizado,DatConclusaoSisTransRealizado,DatPrevisaoIniciobra_x,IdcObraParalisada,IdcUsinaMonitorada,DatMontagemOutorgado,DatConcretagemRealizado,DatConclusaoTorresRealizado,DatInicioObraOutorgado_x,DatConcretagemOutorgado,DatSisTransmissaoRealizado,IdcSituacaoObra,IdcSemPrevisao,DatComissionamentoUGRealizado,DatMonitoramento_x,SigTipoGeracao,DscComercializacaoEnergia_py,DatInicioObraRealizado,DatMontagemRealizado,DthEnvio_x_py,DatConclusaoACL_py,DatValidadeLI_py,DatValidadeLP_py,DatValidadeLO_py,NomSitContratoCCD_py,NomSitContratoCUST_py,NomSituacaoContratoCUSD_py,NomSitContratoCCT_py,ValidadeAmbiental,CondicaoAmbiental,CondicaoConexao_py,CondicaoUso,PPA,criterio_py,dscriterio_py,criterio_novo,NomUsina,NumUgUsina,Previsao_OC,Dat_OC_obrigacao,DatMonitoramento_y,FASE_py,Ind_crono_norm_py,flagOPTeste30dias_py,DatPrevisaoIniciobra_y,DatInicioObraOutorgado_y,DthEnvio_y_py,...,DatValidadeContratoCCD,NomSitContratoCCT_sas,DatAssinaturaContratoCCT,DatValidadeContratoCCT,NomSituacaoContratoCUSD_sas,DatAssinaturaContratoCUSD,DatValidadeContratoCUSD,NomSitContratoCUST_sas,DatAssinaturaContratoCUST,DatValidadeContratoCUST,DatSolicitacaoLP,DatPrevistaEmissaoLP,DatEmissaoLP,DatValidadeLP_sas,NomSituacaoLP,DatSolicitacaoLI,DatPrevistaEmissaoLI,DatEmissaoLI,DatValidadeLI_sas,NomSituacaoLI,DatSolicitacaoLO,DatPrevistaEmissaoLO,DatEmissaoLO,DatValidadeLO_sas,NomSituacaoLO,IdcTipoEntrave_y,DscSolucao_y,DscObservacaoAcesso_y,DscObservacaomercado_y,DscObservacaoAmbiental_y,ValidadeAmbiental_y,CondicaoAmbiental_y,CondicaoConexao_sas,CondicaoUso_y,PPA_y,DscComercializacaoEnergia_sas,DscSistema_y,IdcViabilidade_y,IdcSituacaoObra_y,IdcObraParalisada_y,IdcSemPrevisão_y,criterio_sas,DatPrevistaComercialRapeelMX_y,DscJustificativaPrevisao,DscObservacaoUG_y,DatEmissaoAto_y,criterio_x0020_novo,dscriterio_sas,SAS,OC_diff
74039,28060,NaT,NaT,NaT,NaT,NaT,NaN,NaN,NaT,NaT,NaT,NaT,NaT,NaT,NaN,NaN,NaT,NaT,NaN,NaN,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaT,NaT,NaT,NaN,NaN,NaN,NaT,NaT,NaT,...,2037-04-16,Não_x0020_se_x0020_Aplica,NaT,NaN,Válido,2014-11-24,2037-04-16,Não_x0020_se_x0020_Aplica,NaT,NaN,2019-01-18,NaT,2019-12-13,2022-01-17,Válida,2019-01-18,NaN,2019-12-13,2022-01-17 00:00:00,Válida,2019-01-18,NaN,2019-12-13,2022-01-17,Válida,Não_x0020_há_x0020_pendências,NaN,Informamos_x0020_que_x0020_o_x0020_Parecer_x00...,Por_x0020_limitação_x0020_do_x0020_sistema_x00...,Foram_x0020_utilizadas_x0020_as_x0020_informaç...,2022-01-17,LO,OK,OK,Nenhum,Fora_x0020_do_x0020_ACR,Sistema_x0020_Interligado_x0020_Nacional,Alta,Em_x0020_andamento,Não,Não,0,NaN,Estágio_x0020_das_x0020_obras_x0020_da_x0020_u...,NaN,2007-04-17,0,Usina_x0020_em_x0020_Obras_x0020_em_x0020_anda...,True,NaT


# BIU

In [ ]:
cols = ["DatUGInicioOpComerOutorgado","IdcMonitorada","IdeUsinaOutorga","DatLiberacaoSFGTeste","DatInicioOpTesteOutorgada"]
monitoramentoug = pd.read_parquet(os.path.join(directory,"vmonitoramentoug.gzip"))[cols]

In [ ]:
# vInputCronograma tbm implementado,DATASREAL (LABEL="Idempreendimento" AS Idempreendimento)
cols = ['IdeUsinaOutorga',"DthEnvio","DatRealizacaoIII","DatRealizacaoV","DatRealizacaoXIV","DatRealizacaoXII","DatRealizacaoIV","DatRealizacaoIX","DatRealizacaoXI","DatRealizacaoX","DatRealizacaoVI","DatRealizacaoII","CodCeg","DatRealizacaoXIII","DatRealizacaoVII"]
rename_cols = {'DatRealizacaoIII': 'IO_real','DatRealizacaoXIV': 'Conc_Trans_REAL','DatRealizacaoXII': 'Comiss_Real','DatRealizacaoIX': 'DesvRio_real','DatRealizacaoXI': 'Ench_Real','DatRealizacaoII': 'canteiroReal','DatRealizacaoXIII': 'IOTrans_Real','DatRealizacaoVII': 'ME_Real_conc_eol'}
vcronograma = pd.read_parquet(os.path.join(directory,"vrapeelcronograma.gzip"))[cols].drop_duplicates()[cols].rename(columns=rename_cols)
vcronograma["classe"] = vcronograma.CodCeg.str.slice(3,5)
vcronograma["CC_real"] = pd.to_datetime(pd.NA)
vcronograma.loc[vcronograma.classe == "PH","CC_real"] = vcronograma.DatRealizacaoIV
vcronograma.loc[vcronograma.classe == "CV","CC_real"] = vcronograma.DatRealizacaoV
vcronograma["ME_real"] = vcronograma.DatRealizacaoX
vcronograma.loc[vcronograma.classe == "CV","ME_real"] = vcronograma.DatRealizacaoVI
vcronograma = vcronograma.loc[vcronograma.groupby('IdeUsinaOutorga').DthEnvio.idxmax()]

In [ ]:
ug_rapeel_com_previsao = ug_rapeel.loc[ug_rapeel.DatPrevistaComercial.notna()]
# MAX_Prev_OC_rapeel
ug_rapeel_com_previsao = ug_rapeel_com_previsao.loc[ug_rapeel_com_previsao.groupby('IdeUsinaOutorga').DatPrevistaComercial.idxmax()][['IdeUsinaOutorga','DatPrevistaComercial','DthEnvio']].rename(
    columns = {'DatPrevistaComercial': 'DatPrevistaComercial_MaxUsina'})

In [ ]:
real_cronograma_rapeel = pd.merge(vcronograma,ug_rapeel_com_previsao, on = ['IdeUsinaOutorga','DthEnvio'],how="outer")

In [ ]:
real_cronograma_rapeel

,IdeUsinaOutorga,DthEnvio,IO_real,DatRealizacaoV,Conc_Trans_REAL,Comiss_Real,DatRealizacaoIV,DesvRio_real,Ench_Real,DatRealizacaoX,DatRealizacaoVI,canteiroReal,CodCeg,IOTrans_Real,ME_Real_conc_eol,classe,CC_real,ME_real,DatPrevistaComercial_MaxUsina
0,273,2022-10-09 21:04:17.743,2021-02-15,NaT,NaT,NaT,2021-05-15,2022-03-25,NaT,2021-11-10,NaT,2021-03-15,PCHPHPR000273-9,2022-02-15,NaT,PH,2021-05-15,2021-11-10,2023-11-02
1,324,2023-01-02 08:34:07.347,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,UHEPHRS000324-7,NaT,NaT,PH,NaT,NaT,2026-07-01
2,601,2023-01-03 19:22:43.140,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,PCHPHMG000601-7,NaT,NaT,PH,NaT,NaT,2027-05-01
3,659,2023-01-06 09:37:52.703,NaT,NaT,NaT,NaT,NaT,2016-07-30,2017-07-24,NaT,NaT,NaT,PCHPHSC000659-9,NaT,NaT,PH,NaT,NaT,2023-06-12
4,742,2021-08-09 10:28:28.143,2019-07-01,NaT,2021-05-13,2021-02-16,2020-02-07,2020-10-05,2020-01-03,2020-05-08,NaT,2019-07-01,PCHPHSC000742-0,2020-11-30,NaT,PH,2020-02-07,2020-05-08,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2820,45074,2020-08-04 19:12:09.140,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaN,NaT,NaT,NaN,NaT,NaT,2020-08-24
2821,47242,2022-01-03 15:58:51.557,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaN,NaT,NaT,NaN,NaT,NaT,2024-06-06
2822,47246,2022-01-03 16:06:17.013,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaN,NaT,NaT,NaN,NaT,NaT,2024-04-25
2823,49180,2022-01-03 16:32:22.927,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaN,NaT,NaT,NaN,NaT,NaT,2023-07-17
